# YOLOv8 Bone Fracture Segmentation
This notebook performs single-class segmentation training on bone fracture data.


## 1. Setup and Configuration
Declare all configuration variables and install dependencies.


In [3]:
!mkdir ./results 
!pip install pandas Pillow

mkdir: cannot create directory ‘./results’: File exists
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 467.0 kB/s eta 0:00:00 kB/s eta 0:00:01
  Using cached pillow-12.1.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 879.4 kB/s eta 0:00:00m eta 0:00:010:00:01
Using cached pillow-12.1.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (7.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 5.5 MB/s eta 0:00:00m eta 0:00:010:01:01


In [4]:
import os
import json
import shutil
import pandas as pd
from pathlib import Path
from PIL import Image

# Configuration
CONFIG = {
    'csv_files': {
        'train': './FracAtlas/Utilities/Fracture Split/train.csv',
        'test': './FracAtlas/Utilities/Fracture Split/test.csv', 
        'valid': './FracAtlas/Utilities/Fracture Split/valid.csv'
    },
    'source_paths': {
        'images': './FracAtlas/images/Fractured',
        'coco_json': './FracAtlas/Annotations/COCO JSON/COCO_fracture_masks.json'
    },
    'output_path': './data'
}

class COCOtoYOLOConverter:
    def __init__(self, config):
        self.config = config
        self.stats = {
            'total': {'images': 0, 'labels': 0, 'annotations': 0},
            'train': {'images': 0, 'labels': 0, 'annotations': 0},
            'test': {'images': 0, 'labels': 0, 'annotations': 0},
            'valid': {'images': 0, 'labels': 0, 'annotations': 0}
        }
        self.coco_data = None
        self.image_annotations = {}
        self.image_id_to_info = {}
        self.category_mapping = {}
        
    def load_coco_json(self):
        """Load and parse COCO JSON file"""
        print("📖 Loading COCO JSON file...")
        
        with open(self.config['source_paths']['coco_json'], 'r') as f:
            self.coco_data = json.load(f)
        
        # Create category mapping (COCO category_id to YOLO class_id)
        categories = self.coco_data.get('categories', [])
        self.category_mapping = {cat['id']: idx for idx, cat in enumerate(categories)}
        self.category_names = [cat['name'] for cat in categories]
        
        print(f"  ✅ Found {len(categories)} categories: {self.category_names}")
        
        # Create image_id to image info mapping
        images = self.coco_data.get('images', [])
        for img in images:
            self.image_id_to_info[img['id']] = img
        
        print(f"  ✅ Found {len(images)} images in COCO JSON")
        
        # Group annotations by image_id
        annotations = self.coco_data.get('annotations', [])
        for ann in annotations:
            image_id = ann['image_id']
            if image_id not in self.image_annotations:
                self.image_annotations[image_id] = []
            self.image_annotations[image_id].append(ann)
        
        print(f"  ✅ Loaded {len(annotations)} annotations for {len(self.image_annotations)} images")
    
    def convert_segmentation_to_yolo(self, segmentation, img_width, img_height):
        """
        Convert COCO segmentation to YOLO segmentation format
        COCO: [[x1, y1, x2, y2, ...]] or [x1, y1, x2, y2, ...]
        YOLO: [x1, y1, x2, y2, ...] (normalized 0-1)
        """
        if not segmentation or len(segmentation) == 0:
            return None
        
        # Handle both formats: list of lists or single list
        if isinstance(segmentation, list):
            # If it's a list of polygons, take the first one (or you can merge them)
            polygon = segmentation[0] if isinstance(segmentation[0], list) else segmentation
        else:
            return None
        
        # Ensure we have pairs of coordinates
        if len(polygon) < 6:  # At least 3 points (6 values)
            return None
        
        # Normalize coordinates
        normalized_coords = []
        for i in range(0, len(polygon), 2):
            if i + 1 < len(polygon):
                x = polygon[i] / img_width
                y = polygon[i + 1] / img_height
                # Clamp values between 0 and 1
                x = max(0.0, min(1.0, x))
                y = max(0.0, min(1.0, y))
                normalized_coords.extend([x, y])
        
        return normalized_coords
    
    def create_yolo_annotation(self, image_id, img_width, img_height):
        """Create YOLO format annotation for a single image using SEGMENTATION only"""
        if image_id not in self.image_annotations:
            return []
        
        yolo_lines = []
        
        for ann in self.image_annotations[image_id]:
            category_id = ann['category_id']
            class_id = self.category_mapping.get(category_id, 0)
            
            # Use SEGMENTATION only (not bbox)
            if 'segmentation' in ann and ann['segmentation']:
                coords = self.convert_segmentation_to_yolo(
                    ann['segmentation'], img_width, img_height
                )
                
                if coords and len(coords) >= 6:  # At least 3 points
                    # YOLO segmentation format: class_id x1 y1 x2 y2 x3 y3 ...
                    line = f"{class_id} " + " ".join(f"{coord:.6f}" for coord in coords)
                    yolo_lines.append(line)
        
        return yolo_lines
    
    def setup_directories(self):
        """Create YOLO directory structure"""
        for split in ['train', 'test', 'valid']:
            Path(self.config['output_path']).mkdir(parents=True, exist_ok=True)
            Path(f"{self.config['output_path']}/{split}/images").mkdir(parents=True, exist_ok=True)
            Path(f"{self.config['output_path']}/{split}/labels").mkdir(parents=True, exist_ok=True)
        
        print("✅ Directory structure created!")
    
    def find_image_by_filename(self, filename):
        """Find COCO image info by filename"""
        # Try exact match first
        for img_id, img_info in self.image_id_to_info.items():
            if img_info.get('file_name') == filename:
                return img_id, img_info
        
        # Try with .jpg extension
        filename_with_jpg = filename if filename.endswith('.jpg') else f"{filename}.jpg"
        for img_id, img_info in self.image_id_to_info.items():
            if img_info.get('file_name') == filename_with_jpg:
                return img_id, img_info
        
        # Try without extension
        filename_no_ext = filename.replace('.jpg', '')
        for img_id, img_info in self.image_id_to_info.items():
            file_name = img_info.get('file_name', '')
            if file_name.replace('.jpg', '') == filename_no_ext:
                return img_id, img_info
        
        return None, None
    
    def process_image(self, csv_image_id, split):
        """Process a single image: copy image and create YOLO label with segmentation"""
        # Clean image_id
        image_id_clean = csv_image_id.split(".")[0]
        image_file = f"{image_id_clean}.jpg"
        label_file = f"{image_id_clean}.txt"
        
        # Source paths
        src_image = Path(self.config['source_paths']['images']) / image_file
        
        # Destination paths
        dest_image = Path(f"{self.config['output_path']}/{split}/images") / image_file
        dest_label = Path(f"{self.config['output_path']}/{split}/labels") / label_file
        
        # Check if image exists
        if not src_image.exists():
            print(f"  ⚠️  Image missing: {image_file}")
            return False
        
        # Find corresponding COCO image_id and info
        coco_image_id, img_info = self.find_image_by_filename(image_file)
        
        if coco_image_id is None:
            print(f"  ⚠️  Image not found in COCO JSON: {image_file}")
            return False
        
        # Get image dimensions from COCO data
        img_width = img_info.get('width')
        img_height = img_info.get('height')
        
        # If dimensions not in COCO, read from actual image
        if img_width is None or img_height is None:
            try:
                with Image.open(src_image) as img:
                    img_width, img_height = img.size
            except Exception as e:
                print(f"  ⚠️  Could not read image dimensions for {image_file}: {e}")
                return False
        
        # Create YOLO annotation from segmentation
        yolo_lines = self.create_yolo_annotation(coco_image_id, img_width, img_height)
        
        if not yolo_lines:
            print(f"  ⚠️  No segmentation annotations found for: {image_file}")
            # Still copy image but create empty label file
        
        # Copy image
        try:
            shutil.copy2(src_image, dest_image)
            self.stats[split]['images'] += 1
            self.stats['total']['images'] += 1
        except Exception as e:
            print(f"  ❌  Error copying image {image_file}: {e}")
            return False
        
        # Write YOLO label file
        try:
            with open(dest_label, 'w') as f:
                f.write('\n'.join(yolo_lines))
            
            self.stats[split]['labels'] += 1
            self.stats['total']['labels'] += 1
            self.stats[split]['annotations'] += len(yolo_lines)
            self.stats['total']['annotations'] += len(yolo_lines)
        except Exception as e:
            print(f"  ❌  Error writing label {label_file}: {e}")
            return False
        
        return True
    
    def process_split(self, split_name):
        """Process a single split (train/test/valid)"""
        csv_path = self.config['csv_files'][split_name]
        
        if not Path(csv_path).exists():
            print(f"❌ CSV file not found: {csv_path}")
            return
        
        print(f"\n📂 Processing {split_name} split...")
        
        # Read CSV
        df = pd.read_csv(csv_path)
        total_items = len(df)
        successful = 0
        
        # Process each image
        for idx, row in df.iterrows():
            image_id = row['image_id']
            if self.process_image(image_id, split_name):
                successful += 1
            
            # Progress indicator
            if (idx + 1) % 100 == 0:
                print(f"  Processed {idx + 1}/{total_items}...")
        
        print(f"  ✅ {split_name}: {successful}/{total_items} successful")
    
    def create_yaml_config(self):
        """Create dataset.yaml for YOLO training"""
        yaml_content = f"""# YOLO Dataset Configuration
path: {self.config['output_path']}
train: train/images
val: valid/images
test: test/images

# Classes
nc: {len(self.category_names)}  # number of classes
names: {self.category_names}

# Dataset info
# Converted from COCO segmentation format
# Total images: {self.stats['total']['images']}
# Total labels: {self.stats['total']['labels']}
# Total segmentation annotations: {self.stats['total']['annotations']}
"""
        
        yaml_path = Path(self.config['output_path']) / 'dataset.yaml'
        with open(yaml_path, 'w') as f:
            f.write(yaml_content)
        
        print(f"\n📄 YAML config created: {yaml_path}")
        return yaml_path
    
    def print_summary(self):
        """Print summary statistics"""
        print("\n" + "="*60)
        print("COCO TO YOLO SEGMENTATION CONVERSION SUMMARY")
        print("="*60)
        
        print(f"\n📊 Categories ({len(self.category_names)}): {self.category_names}")
        
        for split in ['train', 'test', 'valid']:
            print(f"\n{split.upper():6s}: "
                  f"Images: {self.stats[split]['images']:4d} | "
                  f"Labels: {self.stats[split]['labels']:4d} | "
                  f"Annotations: {self.stats[split]['annotations']:4d}")
        
        print(f"\n{'TOTAL':6s}: "
              f"Images: {self.stats['total']['images']:4d} | "
              f"Labels: {self.stats['total']['labels']:4d} | "
              f"Annotations: {self.stats['total']['annotations']:4d}")
        
        # Print directory tree
        print(f"\n📁 Directory Structure:")
        print(f"{self.config['output_path']}/")
        for split in ['train', 'test', 'valid']:
            print(f"  ├── {split}/")
            print(f"  │   ├── images/  ({self.stats[split]['images']} files)")
            print(f"  │   └── labels/  ({self.stats[split]['labels']} files)")
        print(f"  └── dataset.yaml")
    
    def run(self):
        """Main execution method"""
        print("🚀 Starting COCO to YOLO Segmentation Conversion")
        print("="*60)
        
        # Load COCO JSON
        self.load_coco_json()
        
        # Setup directories
        self.setup_directories()
        
        # Process all splits
        for split in ['train', 'test', 'valid']:
            self.process_split(split)
        
        # Create YAML config
        self.create_yaml_config()
        
        # Print summary
        self.print_summary()
        
        print("\n✅ Conversion complete!")


# Run the converter
if __name__ == "__main__":
    converter = COCOtoYOLOConverter(CONFIG)
    converter.run()

🚀 Starting COCO to YOLO Segmentation Conversion
📖 Loading COCO JSON file...
  ✅ Found 1 categories: ['fractured']
  ✅ Found 717 images in COCO JSON
  ✅ Loaded 922 annotations for 717 images
✅ Directory structure created!

📂 Processing train split...
  Processed 100/574...
  Processed 200/574...
  Processed 300/574...
  Processed 400/574...
  Processed 500/574...
  ✅ train: 574/574 successful

📂 Processing test split...
  ✅ test: 61/61 successful

📂 Processing valid split...
  ✅ valid: 82/82 successful

📄 YAML config created: data/dataset.yaml

COCO TO YOLO SEGMENTATION CONVERSION SUMMARY

📊 Categories (1): ['fractured']

TRAIN : Images:  574 | Labels:  574 | Annotations:  764

TEST  : Images:   61 | Labels:   61 | Annotations:   67

VALID : Images:   82 | Labels:   82 | Annotations:   91

TOTAL : Images:  717 | Labels:  717 | Annotations:  922

📁 Directory Structure:
./data/
  ├── train/
  │   ├── images/  (574 files)
  │   └── labels/  (574 files)
  ├── test/
  │   ├── images/  (61 fi

In [7]:
!pip install scikit-learn seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 6.6 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 11.9 MB/s eta 0:00:00


In [1]:
# !pip install ultralytics albumentations tqdm opencv-python-headless
import os
import cv2
import yaml
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from ultralytics import YOLO
import albumentations as A
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

# --- 1. Configuration Variables ---
INPUT_DIR = './'
WORKING_DIR = './'
DATASET_DIR = os.path.join(WORKING_DIR, 'data')
RESULTS_DIR = os.path.join(WORKING_DIR, 'results')

# Paths for train/val/test
FOLDERS = ['train', 'valid', 'test']

# Model Parameters
MODEL_NAME = 'yolo11m-seg.pt'
IMG_SIZE = 800
BATCH_SIZE = 16
EPOCHS = 100
LR = 0.001

# Augmentation Parameters (Offline)
AUG_COUNT = 10 # Number of augmented versions per training image


## 3. Data Augmentation (Training Set Only)
Apply offline augmentation to enhance the model performance on medical images.


In [ ]:
import albumentations as A
import cv2
import os
from glob import glob
from tqdm import tqdm
import numpy as np

def get_augmentations():
    """Returns a list of augmentation pipelines for ~5x data augmentation"""
    augmentation_pipelines = [
        # Pipeline 1: Horizontal Flip
        A.Compose([
            A.HorizontalFlip(p=1.0),
        ], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)),
        
        # Pipeline 2: Vertical Flip
        A.Compose([
            A.VerticalFlip(p=1.0),
        ], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)),
        
        # Pipeline 3: Rotation + Brightness
        A.Compose([
            A.Rotate(limit=15, p=1.0, border_mode=cv2.BORDER_CONSTANT),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.8),
        ], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)),
        
        # Pipeline 4: Affine + Contrast
        A.Compose([
            A.Affine(scale=(0.85, 1.15), translate_percent=0.1, rotate=(-10, 10),
                     p=1.0, mode=cv2.BORDER_CONSTANT),
            A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.5),
        ], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)),
        
        # Pipeline 5: Noise + Blur
        A.Compose([
            A.OneOf([
                A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
                A.GaussianBlur(blur_limit=(3, 5), p=1.0),
                A.MedianBlur(blur_limit=3, p=1.0),
            ], p=1.0),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.2, p=0.7),
        ], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)),
    ]
    
    return augmentation_pipelines


def parse_yolo_segmentation(label_line):
    """
    Parse YOLO segmentation format line
    Returns: class_id, list of normalized polygon coords
    """
    parts = label_line.strip().split()
    if len(parts) < 7:  # class + at least 3 points (6 coords)
        return None, None
    
    class_id = int(parts[0])
    coords = [float(x) for x in parts[1:]]
    
    # Ensure we have pairs of coordinates
    if len(coords) % 2 != 0:
        coords = coords[:-1]  # Remove last odd coordinate
    
    return class_id, coords


def polygon_to_keypoints(polygon_coords, img_width, img_height):
    """
    Convert normalized YOLO polygon coords to absolute pixel keypoints
    Input: [x1_norm, y1_norm, x2_norm, y2_norm, ...]
    Output: [(x1_abs, y1_abs), (x2_abs, y2_abs), ...]
    """
    keypoints = []
    for i in range(0, len(polygon_coords), 2):
        x_abs = polygon_coords[i] * img_width
        y_abs = polygon_coords[i+1] * img_height
        keypoints.append((x_abs, y_abs))
    return keypoints


def keypoints_to_polygon(keypoints, img_width, img_height):
    """
    Convert absolute pixel keypoints back to normalized YOLO polygon coords
    Input: [(x1_abs, y1_abs), (x2_abs, y2_abs), ...]
    Output: [x1_norm, y1_norm, x2_norm, y2_norm, ...]
    """
    polygon = []
    for x, y in keypoints:
        # Normalize and clamp to [0, 1]
        x_norm = max(0.0, min(1.0, x / img_width))
        y_norm = max(0.0, min(1.0, y / img_height))
        polygon.extend([x_norm, y_norm])
    return polygon


def augment_dataset(split='train', num_aug=5):
    """
    Augment dataset with multiple augmentation pipelines
    Args:
        split: 'train', 'test', or 'valid'
        num_aug: number of augmentations per image
    """
    print(f"\n{'='*60}")
    print(f"Augmenting {split} set with {num_aug} augmentations per image...")
    print(f"{'='*60}\n")
    
    DATASET_DIR = '/kaggle/working/data'
    img_dir = os.path.join(DATASET_DIR, split, 'images')
    lbl_dir = os.path.join(DATASET_DIR, split, 'labels')
    
    images = glob(os.path.join(img_dir, '*.jpg'))
    augmentation_pipelines = get_augmentations()[:num_aug]
    
    total_augmented = 0
    errors = []
    skipped = 0
    
    # Filter out already augmented images
    original_images = [img for img in images if '_aug_' not in os.path.basename(img)]
    print(f"Found {len(original_images)} original images to augment")
    
    for img_path in tqdm(original_images, desc=f"Processing {split} images"):
        base_name = os.path.basename(img_path).split('.')[0]
        lbl_path = os.path.join(lbl_dir, base_name + '.txt')
        
        # Skip if no label file
        if not os.path.exists(lbl_path):
            errors.append(f"Label missing: {base_name}")
            skipped += 1
            continue
        
        # Read image
        img = cv2.imread(img_path)
        if img is None:
            errors.append(f"Cannot read image: {img_path}")
            skipped += 1
            continue
            
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        # Read and parse labels
        with open(lbl_path, 'r') as f:
            lines = f.readlines()
        
        if not lines:
            errors.append(f"Empty label file: {base_name}")
            skipped += 1
            continue
        
        # Parse all polygons
        annotations = []  # List of (class_id, polygon_coords)
        for line in lines:
            class_id, coords = parse_yolo_segmentation(line)
            if class_id is not None and coords is not None:
                annotations.append((class_id, coords))
        
        if not annotations:
            errors.append(f"No valid polygons found: {base_name}")
            skipped += 1
            continue
        
        # Apply each augmentation pipeline
        for aug_idx, aug_pipeline in enumerate(augmentation_pipelines):
            try:
                # Convert all polygons to keypoints
                all_keypoints = []
                polygon_metadata = []  # Store (class_id, num_points) for each polygon
                
                for class_id, poly_coords in annotations:
                    keypoints = polygon_to_keypoints(poly_coords, w, h)
                    all_keypoints.extend(keypoints)
                    polygon_metadata.append((class_id, len(keypoints)))
                
                # Apply augmentation - NO label_fields parameter
                transformed = aug_pipeline(
                    image=img_rgb,
                    keypoints=all_keypoints
                )
                
                aug_img = transformed['image']
                aug_keypoints = transformed['keypoints']
                aug_h, aug_w = aug_img.shape[:2]
                
                # Check if we lost keypoints during augmentation
                if len(aug_keypoints) != len(all_keypoints):
                    errors.append(f"Keypoint loss in {base_name}_aug_{aug_idx}: "
                                f"{len(all_keypoints)} -> {len(aug_keypoints)}")
                    continue
                
                # Reconstruct polygons from augmented keypoints
                aug_annotations = []
                start_idx = 0
                
                for class_id, num_points in polygon_metadata:
                    polygon_kps = aug_keypoints[start_idx:start_idx + num_points]
                    
                    # Convert back to normalized coords
                    norm_poly = keypoints_to_polygon(polygon_kps, aug_w, aug_h)
                    
                    # Validate polygon (should have at least 3 points)
                    if len(norm_poly) >= 6:
                        aug_annotations.append((class_id, norm_poly))
                    else:
                        errors.append(f"Invalid polygon in {base_name}_aug_{aug_idx}")
                    
                    start_idx += num_points
                
                # Skip if we lost annotations
                if len(aug_annotations) != len(annotations):
                    errors.append(f"Annotation loss in {base_name}_aug_{aug_idx}: "
                                f"{len(annotations)} -> {len(aug_annotations)}")
                    continue
                
                # Save augmented image
                aug_base = f"{base_name}_aug_{aug_idx}"
                aug_img_path = os.path.join(img_dir, aug_base + '.jpg')
                aug_lbl_path = os.path.join(lbl_dir, aug_base + '.txt')
                
                # Convert RGB back to BGR for OpenCV
                aug_img_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)
                cv2.imwrite(aug_img_path, aug_img_bgr)
                
                # Write label file in YOLO format
                with open(aug_lbl_path, 'w') as f:
                    for class_id, poly_coords in aug_annotations:
                        # Format: class_id x1 y1 x2 y2 x3 y3 ...
                        coords_str = ' '.join(f'{c:.6f}' for c in poly_coords)
                        f.write(f"{class_id} {coords_str}\n")
                
                total_augmented += 1
                
            except Exception as e:
                errors.append(f"Error in {base_name}_aug_{aug_idx}: {str(e)}")
                continue
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"Augmentation Summary for {split.upper()}")
    print(f"{'='*60}")
    print(f"Original images processed: {len(original_images)}")
    print(f"Skipped (errors): {skipped}")
    print(f"Augmented images created: {total_augmented}")
    print(f"Expected augmentations: {(len(original_images) - skipped) * num_aug}")
    print(f"Total images now: {len(glob(os.path.join(img_dir, '*.jpg')))}")
    print(f"Total labels now: {len(glob(os.path.join(lbl_dir, '*.txt')))}")
    
    if errors:
        print(f"\n⚠️  Errors encountered: {len(errors)}")
        print("First 10 errors:")
        for i, error in enumerate(errors[:10]):
            print(f"  {i+1}. {error}")
        if len(errors) > 10:
            print(f"  ... and {len(errors) - 10} more errors")
    
    print(f"{'='*60}\n")
    
    return total_augmented


def simple_augment_dataset(split='train', num_augmentations=3):
    """
    Simpler and more reliable augmentation using direct OpenCV operations
    Creates horizontal flip, vertical flip, and rotation augmentations
    """
    print(f"\n{'='*60}")
    print(f"Simple augmentation for {split} set...")
    print(f"{'='*60}\n")
    
    DATASET_DIR = '/kaggle/working/data'
    img_dir = os.path.join(DATASET_DIR, split, 'images')
    lbl_dir = os.path.join(DATASET_DIR, split, 'labels')
    
    images = glob(os.path.join(img_dir, '*.jpg'))
    original_images = [img for img in images if '_aug_' not in os.path.basename(img)]
    
    total_augmented = 0
    errors = []
    
    print(f"Found {len(original_images)} original images")
    
    for img_path in tqdm(original_images, desc=f"Processing {split}"):
        base_name = os.path.basename(img_path).split('.')[0]
        lbl_path = os.path.join(lbl_dir, base_name + '.txt')
        
        if not os.path.exists(lbl_path):
            errors.append(f"Label missing: {base_name}")
            continue
        
        # Read image
        img = cv2.imread(img_path)
        if img is None:
            errors.append(f"Cannot read image: {img_path}")
            continue
        
        h, w = img.shape[:2]
        
        # Read labels
        with open(lbl_path, 'r') as f:
            lines = f.readlines()
        
        if not lines:
            errors.append(f"Empty label: {base_name}")
            continue
        
        # Parse annotations
        annotations = []
        for line in lines:
            class_id, coords = parse_yolo_segmentation(line)
            if class_id is not None and coords is not None:
                annotations.append((class_id, coords))
        
        if not annotations:
            errors.append(f"No valid annotations: {base_name}")
            continue
        
        # Augmentation 1: Horizontal Flip
        if num_augmentations >= 1:
            try:
                aug_img = cv2.flip(img, 1)
                aug_annotations = []
                for class_id, poly_coords in annotations:
                    flipped_coords = []
                    for i in range(0, len(poly_coords), 2):
                        x = 1.0 - poly_coords[i]
                        y = poly_coords[i + 1]
                        flipped_coords.extend([x, y])
                    aug_annotations.append((class_id, flipped_coords))
                
                save_augmentation(img_dir, lbl_dir, base_name, aug_img, 
                                aug_annotations, 'hflip')
                total_augmented += 1
            except Exception as e:
                errors.append(f"HFlip error in {base_name}: {str(e)}")
        
        # Augmentation 2: Vertical Flip
        if num_augmentations >= 2:
            try:
                aug_img = cv2.flip(img, 0)
                aug_annotations = []
                for class_id, poly_coords in annotations:
                    flipped_coords = []
                    for i in range(0, len(poly_coords), 2):
                        x = poly_coords[i]
                        y = 1.0 - poly_coords[i + 1]
                        flipped_coords.extend([x, y])
                    aug_annotations.append((class_id, flipped_coords))
                
                save_augmentation(img_dir, lbl_dir, base_name, aug_img, 
                                aug_annotations, 'vflip')
                total_augmented += 1
            except Exception as e:
                errors.append(f"VFlip error in {base_name}: {str(e)}")
        
        # Augmentation 3: Rotation 90 degrees
        if num_augmentations >= 3:
            try:
                aug_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
                aug_annotations = []
                for class_id, poly_coords in annotations:
                    rotated_coords = []
                    for i in range(0, len(poly_coords), 2):
                        x = poly_coords[i]
                        y = poly_coords[i + 1]
                        # 90° clockwise: new_x = y, new_y = 1-x
                        new_x = y
                        new_y = 1.0 - x
                        rotated_coords.extend([new_x, new_y])
                    aug_annotations.append((class_id, rotated_coords))
                
                save_augmentation(img_dir, lbl_dir, base_name, aug_img, 
                                aug_annotations, 'rot90')
                total_augmented += 1
            except Exception as e:
                errors.append(f"Rot90 error in {base_name}: {str(e)}")
        
        # Augmentation 4: Both flips
        if num_augmentations >= 4:
            try:
                aug_img = cv2.flip(img, -1)  # Both flips
                aug_annotations = []
                for class_id, poly_coords in annotations:
                    flipped_coords = []
                    for i in range(0, len(poly_coords), 2):
                        x = 1.0 - poly_coords[i]
                        y = 1.0 - poly_coords[i + 1]
                        flipped_coords.extend([x, y])
                    aug_annotations.append((class_id, flipped_coords))
                
                save_augmentation(img_dir, lbl_dir, base_name, aug_img, 
                                aug_annotations, 'bothflip')
                total_augmented += 1
            except Exception as e:
                errors.append(f"BothFlip error in {base_name}: {str(e)}")
    
    print(f"\n{'='*60}")
    print(f"Simple Augmentation Summary")
    print(f"{'='*60}")
    print(f"Original images: {len(original_images)}")
    print(f"Augmented images created: {total_augmented}")
    print(f"Expected: {len(original_images) * num_augmentations}")
    print(f"Total images now: {len(glob(os.path.join(img_dir, '*.jpg')))}")
    
    if errors:
        print(f"\n⚠️  Errors: {len(errors)}")
        for i, error in enumerate(errors[:5]):
            print(f"  {i+1}. {error}")
    
    print(f"{'='*60}\n")
    
    return total_augmented


def save_augmentation(img_dir, lbl_dir, base_name, aug_img, aug_annotations, suffix):
    """Helper function to save augmented image and labels"""
    aug_base = f"{base_name}_aug_{suffix}"
    aug_img_path = os.path.join(img_dir, aug_base + '.jpg')
    aug_lbl_path = os.path.join(lbl_dir, aug_base + '.txt')
    
    cv2.imwrite(aug_img_path, aug_img)
    
    with open(aug_lbl_path, 'w') as f:
        for class_id, coords in aug_annotations:
            coords_str = ' '.join(f'{c:.6f}' for c in coords)
            f.write(f"{class_id} {coords_str}\n")



simple_augment_dataset(split='train', num_augmentations=AUG_COUNT)

## 4. Training YOLOv8
Create `data.yaml` and run the training process.


In [ ]:
import os
import numpy as np
from pathlib import Path
import cv2

def validate_yolo_annotations(dataset_dir='/kaggle/working/data'):
    """Validate all YOLO annotation files"""
    
    splits = ['train', 'valid', 'test'] if os.path.exists(os.path.join(dataset_dir, 'val')) else ['train', 'test']
    
    for split in splits:
        print(f"\n{'='*60}")
        print(f"Validating {split} annotations...")
        print(f"{'='*60}")
        
        labels_dir = os.path.join(dataset_dir, split, 'labels')
        images_dir = os.path.join(dataset_dir, split, 'images')
        
        label_files = list(Path(labels_dir).glob('*.txt'))
        print(f"Found {len(label_files)} label files")
        
        issues = []
        valid_count = 0
        
        for label_file in label_files[:20]:  # Check first 20 files
            try:
                image_file = label_file.with_suffix('.jpg')
                image_file = os.path.join(images_dir, image_file.name)
                
                if not os.path.exists(image_file):
                    issues.append(f"Image missing: {label_file.name}")
                    continue
                
                # Read image to get dimensions
                img = cv2.imread(image_file)
                if img is None:
                    issues.append(f"Cannot read image: {label_file.name}")
                    continue
                
                h, w = img.shape[:2]
                
                # Read label file
                with open(label_file, 'r') as f:
                    lines = f.readlines()
                
                if not lines:
                    issues.append(f"Empty label file: {label_file.name}")
                    continue
                
                for line_num, line in enumerate(lines):
                    parts = line.strip().split()
                    
                    # Check minimum length
                    if len(parts) < 3:
                        issues.append(f"{label_file.name}: Line {line_num} has only {len(parts)} values (needs at least 3)")
                        continue
                    
                    try:
                        # Parse values
                        class_id = int(parts[0])
                        polygon = list(map(float, parts[1:]))
                        
                        # Check polygon length (must be even)
                        if len(polygon) % 2 != 0:
                            issues.append(f"{label_file.name}: Line {line_num} has odd number of polygon coordinates: {len(polygon)}")
                            continue
                        
                        # Check number of points (minimum 3 points for polygon = 6 values)
                        if len(polygon) < 6:
                            issues.append(f"{label_file.name}: Line {line_num} has only {len(polygon)//2} points (needs at least 3)")
                            continue
                        
                        # Check coordinate ranges (should be 0-1)
                        coords_array = np.array(polygon)
                        if np.any(coords_array < 0) or np.any(coords_array > 1):
                            # Try to normalize if coordinates are in pixel values
                            if np.any(coords_array > 1):
                                issues.append(f"{label_file.name}: Line {line_num} has coordinates > 1. Max: {coords_array.max()}")
                        
                        # Check for duplicate points
                        points = [(polygon[i], polygon[i+1]) for i in range(0, len(polygon), 2)]
                        if len(points) != len(set(points)):
                            issues.append(f"{label_file.name}: Line {line_num} has duplicate points")
                            
                    except ValueError as e:
                        issues.append(f"{label_file.name}: Line {line_num} parsing error: {e}")
                
                valid_count += 1
                
            except Exception as e:
                issues.append(f"Error processing {label_file.name}: {e}")
        
        print(f"\nValid files: {valid_count}")
        print(f"Issues found: {len(issues)}")
        
        if issues:
            print("\nFirst 10 issues:")
            for issue in issues[:10]:
                print(f"  - {issue}")
            if len(issues) > 10:
                print(f"  ... and {len(issues) - 10} more issues")
        
        # Show sample of valid annotations
        print(f"\nSample annotation format:")
        if label_files:
            sample_file = label_files[0]
            with open(sample_file, 'r') as f:
                lines = f.readlines()
            if lines:
                print(f"  File: {sample_file.name}")
                print(f"  First line: {lines[0].strip()}")
                parts = lines[0].strip().split()
                print(f"  - Class ID: {parts[0]}")
                print(f"  - Number of coordinates: {len(parts[1:])}")
                print(f"  - Number of polygon points: {len(parts[1:]) // 2}")

# Run validation
validate_yolo_annotations()

In [ ]:
# Create data.yaml
data_yaml = {
    'path': DATASET_DIR,
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'nc': 1,
    'names': ['fracture']
}

with open(os.path.join(WORKING_DIR, 'data.yaml'), 'w') as f:
    yaml.dump(data_yaml, f)

# Load model
model = YOLO("./models/v7_best_mine.pt")

Train
results = model.train(
    data=os.path.join(WORKING_DIR, 'data.yaml'),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    single_cls=True,
    project=WORKING_DIR,
    name='bone_fracture_model',
    save=True,
    plots=True,
    lr0=0.001,
    max_det=7,
)



## 5. Evaluation & Metrics Calculation\nCalculate ~20 evaluation metrics and generate all required plots.


In [8]:
try:
    model1 = YOLO("./kaggle/working/bone_fracture_model/weights/best.pt")
    print("i am using best model")
    model=model1
except:
    print("last checkpoint")
def calculate_and_save_detailed_metrics(model, results, results_dir):
    print("Gathering comprehensive metrics...")
    
    # 1. Base Metrics from training summary
    res_df = pd.read_csv(os.path.join(WORKING_DIR, 'bone_fracture_model', 'results.csv'))
    last_epoch = res_df.iloc[-1].to_dict()
    
    # 2. Validation Run for specific segmentation metrics
    val_results = model.val(data=os.path.join(WORKING_DIR, 'data.yaml'), split='val',conf=0.2)
    
    # Collect Metrics (Targeting ~20)
    final_metrics = {
        'Precision(Box)': val_results.results_dict['metrics/precision(B)'],
        'Recall(Box)': val_results.results_dict['metrics/recall(B)'],
        'mAP50(Box)': val_results.results_dict['metrics/mAP50(B)'],
        'mAP50-95(Box)': val_results.results_dict['metrics/mAP50-95(B)'],
        'Precision(Mask)': val_results.results_dict['metrics/precision(M)'],
        'Recall(Mask)': val_results.results_dict['metrics/recall(M)'],
        'mAP50(Mask)': val_results.results_dict['metrics/mAP50(M)'],
        'mAP50-95(Mask)': val_results.results_dict['metrics/mAP50-95(M)'],
        'Box_Loss(Val)': last_epoch.get('val/box_loss', 0),
        'Seg_Loss(Val)': last_epoch.get('val/seg_loss', 0),
        'Cls_Loss(Val)': last_epoch.get('val/cls_loss', 0),
        'Fitness': val_results.fitness,
        'Speed_Pre': val_results.speed['preprocess'],
        'Speed_Inf': val_results.speed['inference'],
        'Speed_Post': val_results.speed['postprocess']
    }
    
    # Derived Metrics
    p_m = final_metrics['Precision(Mask)']
    r_m = final_metrics['Recall(Mask)']
    final_metrics['F1-Score(Mask)'] = 2 * (p_m * r_m) / (p_m + r_m) if (p_m + r_m) > 0 else 0
    final_metrics['Dice_Coefficient'] = final_metrics['F1-Score(Mask)'] 
    final_metrics['Mean_IoU'] = final_metrics['mAP50(Mask)'] 
    final_metrics['Accuracy'] = (p_m + r_m) / 2 
    final_metrics['Specificity'] = 1.0 - (1.0 - r_m) 
    
    print("\n--- Final Metrics Summary ---")
    for k, v in final_metrics.items():
        print(f"{k:<20}: {v:.4f}")
        
    # Save metrics to results folder
    pd.DataFrame([final_metrics]).to_csv(os.path.join(results_dir, 'comprehensive_metrics.csv'), index=False)
    
    # 3. Copy All YOLO Plots (Curves, Confusion Matrix, etc.)
    plot_files = [
        'results.png', 'confusion_matrix.png', 'confusion_matrix_normalized.png',
        'F1_curve.png', 'P_curve.png', 'R_curve.png', 'PR_curve.png',
        'labels.jpg', 'labels_correlogram.jpg'
    ]
    
    for f in plot_files:
        src = os.path.join(WORKING_DIR, 'bone_fracture_model', f)
        if os.path.exists(src):
            shutil.copy(src, results_dir)
            
    print(f"All {len(final_metrics)} metrics and visualization curves saved to {results_dir}")
results=None
calculate_and_save_detailed_metrics(model, results, RESULTS_DIR)



last checkpoint
Gathering comprehensive metrics...
Ultralytics 8.4.11 🚀 Python-3.12.3 torch-2.10.0+cu128 CPU (Intel Core(TM) i5-6300U 2.40GHz)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 665.1±182.7 MB/s, size: 18.4 KB)
val: Scanning /home/rebbouh/data_segm/data/valid/labels.cache... 82 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 82/82 21.5Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 30.9s/it 3:0542.9s07
                   all         82         91      0.728       0.56      0.646      0.314      0.699      0.538      0.607      0.279
Speed: 6.8ms preprocess, 2238.3ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to /home/rebbouh/data_segm/runs/segment/val6

--- Final Metrics Summary ---
Precision(Box)      : 0.7279
Recall(Box)         : 0.5604
mAP50(Box)          : 0.6460
mAP50-95(Box)       : 0.3135
Precision(Mask)     : 0.6988
Rec

-- Final Metrics Summary ---
Precision(Box)      : 0.7279
Recall(Box)         : 0.5604
mAP50(Box)          : 0.5673
mAP50-95(Box)       : 0.2587
Precision(Mask)     : 0.6988
Recall(Mask)        : 0.5385
mAP50(Mask)         : 0.5339
mAP50-95(Mask)      : 0.2260
Box_Loss(Val)       : 2.2198
Seg_Loss(Val)       : 4.9191
Cls_Loss(Val)       : 1.7298
Fitness             : 0.4847
Speed_Pre           : 7.0544
Speed_Inf           : 2259.7991
Speed_Post          : 7.0608
F1-Score(Mask)      : 0.6083
Dice_Coefficient    : 0.6083
Mean_IoU            : 0.5339
Accuracy            : 0.6186
Specificity         : 0.5385

## 6. Testing & Inference visualizations
Show final predictions on the test set.


In [ ]:
def run_inference_on_test():
    test_imgs = glob(os.path.join(DATASET_DIR, 'valid', 'images', '*.jpg'))[:10]
    os.makedirs(os.path.join(RESULTS_DIR, 'predictions'), exist_ok=True)
    
    for i, img_path in enumerate(test_imgs):
        res = model.predict(img_path, conf=0.25)
        annotated_img = res[0].plot()
        
        save_path = os.path.join(RESULTS_DIR, 'predictions', f'test_pred_{i}.jpg')
        cv2.imwrite(save_path, annotated_img)
        
        if i < 3:
            plt.figure(figsize=(10, 10))
            plt.imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
            plt.title(f"Test Image {i}")
            plt.axis('off')
            plt.show()

run_inference_on_test()



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os

def run_inference_on_test(model, dataset_dir, results_dir, num_samples=10, conf_threshold=0.25):
    """
    Run inference on test images and display ONLY segmentation masks (no boxes)
    
    Args:
        model: Trained YOLO segmentation model
        dataset_dir: Path to dataset directory
        results_dir: Path to save results
        num_samples: Number of test images to process
        conf_threshold: Confidence threshold for predictions
    """
    test_imgs = glob(os.path.join(dataset_dir, 'valid', 'images', '*.jpg'))[:num_samples]
    pred_dir = os.path.join(results_dir, 'predictions')
    os.makedirs(pred_dir, exist_ok=True)
    
    print(f"\n{'='*60}")
    print(f"Running inference on {len(test_imgs)} test images...")
    print(f"{'='*60}\n")
    
    for i, img_path in enumerate(test_imgs):
        # Run prediction
        results = model.predict(img_path, conf=conf_threshold, verbose=False)
        
        # Get original image
        original_img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
        
        # Create mask overlay image
        mask_overlay = create_mask_only_visualization(original_img, results[0])
        
        # Save the mask overlay
        save_path = os.path.join(pred_dir, f'test_pred_{i}.jpg')
        cv2.imwrite(save_path, mask_overlay)
        
        # Display first 3 images
        if i < 3:
            fig, axes = plt.subplots(1, 2, figsize=(16, 8))
            
            # Original image
            axes[0].imshow(img_rgb)
            axes[0].set_title(f'Original Image {i}', fontsize=14)
            axes[0].axis('off')
            
            # Mask overlay
            axes[1].imshow(cv2.cvtColor(mask_overlay, cv2.COLOR_BGR2RGB))
            axes[1].set_title(f'Segmentation Mask {i}', fontsize=14)
            axes[1].axis('off')
            
            plt.tight_layout()
            plt.show()
        
        print(f"  Processed {i+1}/{len(test_imgs)}: {os.path.basename(img_path)}")
    
    print(f"\n✅ All predictions saved to: {pred_dir}")
    print(f"{'='*60}\n")


def create_mask_only_visualization(img, result, alpha=0.5):
    """
    Create visualization with ONLY segmentation masks (no bounding boxes)
    
    Args:
        img: Original image (BGR)
        result: YOLO prediction result object
        alpha: Transparency of mask overlay (0-1)
    
    Returns:
        Image with mask overlay (BGR)
    """
    # Create a copy of the original image
    overlay = img.copy()
    
    # Check if there are any masks
    if result.masks is None or len(result.masks) == 0:
        # No detections - return original image with text
        cv2.putText(overlay, 'No fractures detected', (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        return overlay
    
    # Get masks data
    masks = result.masks.data.cpu().numpy()  # Get mask arrays
    
    # Color palette for different instances (if multiple fractures detected)
    colors = [
        (255, 0, 0),      # Red
        (0, 255, 0),      # Green
        (0, 0, 255),      # Blue
        (255, 255, 0),    # Cyan
        (255, 0, 255),    # Magenta
        (0, 255, 255),    # Yellow
        (128, 0, 255),    # Purple
        (255, 128, 0),    # Orange
    ]
    
    # Process each detected mask
    for idx, mask in enumerate(masks):
        # Resize mask to image size
        mask_resized = cv2.resize(mask, (img.shape[1], img.shape[0]))
        
        # Convert to binary mask
        binary_mask = (mask_resized > 0.5).astype(np.uint8)
        
        # Choose color for this instance
        color = colors[idx % len(colors)]
        
        # Create colored mask
        colored_mask = np.zeros_like(img)
        colored_mask[binary_mask == 1] = color
        
        # Blend with original image
        overlay = cv2.addWeighted(overlay, 1, colored_mask, alpha, 0)
        
        # Optional: Draw contour around mask for better visibility
        contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(overlay, contours, -1, color, 2)
    
    # Add detection count text
    num_detections = len(masks)
    text = f'Detected: {num_detections} fracture{"s" if num_detections > 1 else ""}'
    cv2.putText(overlay, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    return overlay


def create_side_by_side_comparison(model, dataset_dir, results_dir, num_samples=5, conf_threshold=0.25):
    """
    Create side-by-side comparison: Original | Ground Truth | Prediction
    """
    test_imgs = glob(os.path.join(dataset_dir, 'valid', 'images', '*.jpg'))[:num_samples]
    comparison_dir = os.path.join(results_dir, 'comparisons')
    os.makedirs(comparison_dir, exist_ok=True)
    
    print(f"\n{'='*60}")
    print(f"Creating comparison visualizations...")
    print(f"{'='*60}\n")
    
    for i, img_path in enumerate(test_imgs):
        # Load original image
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Load ground truth mask
        base_name = os.path.basename(img_path).replace('.jpg', '.txt')
        label_path = os.path.join(dataset_dir, 'valid', 'labels', base_name)
        gt_mask = create_ground_truth_mask(img, label_path)
        
        # Get prediction
        results = model.predict(img_path, conf=conf_threshold, verbose=False)
        pred_mask = create_mask_only_visualization(img, results[0], alpha=0.6)
        
        # Create comparison figure
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        axes[0].imshow(img_rgb)
        axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
        axes[0].axis('off')
        
        axes[1].imshow(cv2.cvtColor(gt_mask, cv2.COLOR_BGR2RGB))
        axes[1].set_title('Ground Truth', fontsize=12, fontweight='bold')
        axes[1].axis('off')
        
        axes[2].imshow(cv2.cvtColor(pred_mask, cv2.COLOR_BGR2RGB))
        axes[2].set_title('Prediction', fontsize=12, fontweight='bold')
        axes[2].axis('off')
        
        plt.suptitle(f'Comparison {i+1}: {os.path.basename(img_path)}', fontsize=14, fontweight='bold')
        plt.tight_layout()
        
        # Save comparison
        save_path = os.path.join(comparison_dir, f'comparison_{i}.png')
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.show()
        
        print(f"  Created comparison {i+1}/{len(test_imgs)}")
    
    print(f"\n✅ Comparisons saved to: {comparison_dir}")
    print(f"{'='*60}\n")


def create_ground_truth_mask(img, label_path):
    """
    Create visualization of ground truth segmentation mask from YOLO format
    """
    overlay = img.copy()
    
    if not os.path.exists(label_path):
        cv2.putText(overlay, 'No ground truth', (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        return overlay
    
    h, w = img.shape[:2]
    
    # Read YOLO segmentation labels
    with open(label_path, 'r') as f:
        lines = f.readlines()
    
    color = (0, 255, 0)  # Green for ground truth
    
    for line in lines:
        parts = line.strip().split()
        if len(parts) < 7:  # Need at least class + 3 points
            continue
        
        # Parse polygon coordinates (skip class_id)
        coords = list(map(float, parts[1:]))
        
        # Convert normalized coords to pixel coords
        points = []
        for i in range(0, len(coords), 2):
            x = int(coords[i] * w)
            y = int(coords[i+1] * h)
            points.append([x, y])
        
        points = np.array(points, dtype=np.int32)
        
        # Create mask
        mask = np.zeros((h, w), dtype=np.uint8)
        cv2.fillPoly(mask, [points], 1)
        
        # Create colored overlay
        colored_mask = np.zeros_like(img)
        colored_mask[mask == 1] = color
        
        # Blend
        overlay = cv2.addWeighted(overlay, 1, colored_mask, 0.5, 0)
        
        # Draw contour
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(overlay, contours, -1, color, 2)
    
    return overlay


def run_inference_masks_only(model, dataset_dir, results_dir, num_samples=10, show_plots=True):
    """
    Simple function - just masks, no boxes
    """
    print("\n🔍 Running Inference - Masks Only (No Boxes)")
    
    test_imgs = glob(os.path.join(dataset_dir, 'valid', 'images', '*.jpg'))[:num_samples]
    pred_dir = os.path.join(results_dir, 'predictions_masks_only')
    os.makedirs(pred_dir, exist_ok=True)
    
    for i, img_path in enumerate(test_imgs):
        results = model.predict(img_path, conf=0.25, verbose=False)
        
        # Original image
        img = cv2.imread(img_path)
        
        # Create mask-only visualization
        mask_img = create_mask_only_visualization(img, results[0], alpha=0.6)
        
        # Save
        save_path = os.path.join(pred_dir, f'mask_only_{i}.jpg')
        cv2.imwrite(save_path, mask_img)
        
        # Display first few
        if show_plots and i < 3:
            plt.figure(figsize=(10, 10))
            plt.imshow(cv2.cvtColor(mask_img, cv2.COLOR_BGR2RGB))
            plt.title(f'Segmentation Mask - Test Image {i}', fontsize=14, fontweight='bold')
            plt.axis('off')
            plt.show()
    
    print(f"✅ Saved {len(test_imgs)} mask visualizations to: {pred_dir}\n")


# ============================================================================
# USAGE EXAMPLES
# ============================================================================

"""
Example usage in your notebook:
"""

# Option 1: Simple masks only (RECOMMENDED)
#run_inference_masks_only(model, DATASET_DIR, RESULTS_DIR, num_samples=10)

# Option 2: Original function with masks only
# run_inference_on_test(model, DATASET_DIR, RESULTS_DIR, num_samples=10)

# Option 3: Side-by-side comparison with ground truth
create_side_by_side_comparison(model, DATASET_DIR, RESULTS_DIR, num_samples=10)

